In [1]:
import sys
import pprint
import datetime
from tlslite.utils import keyfactory
import pymongo
import pprint
import math
import pandas as pd
import numpy as np
from plotly import tools
import plotly.graph_objs as go
import plotly.offline as py
import os

py.init_notebook_mode(connected=True)

In [2]:
client = pymongo.MongoClient('18.8.8.119')
cydb = client.mytest

issues = pd.DataFrame(list(cydb.useract_info.find()))
len(issues)

97435

In [3]:
del issues['_id']

In [4]:
valid_data = issues.loc[issues['IMEI'].apply(lambda x:len(x.split(';')) == 2)]

In [5]:
imeis = len(valid_data['IMEI'].unique())

In [70]:
imeis

256

In [6]:
total_appclick = valid_data[valid_data.name == 'appclick']

In [7]:
appclick = total_appclick.groupby(['attrs_packagename']).size().sort_values().truediv(len(total_appclick))

In [12]:
trace = go.Bar(y=appclick.index, x=appclick.values, orientation = 'h',
                  text = appclick.apply(lambda x:'{:.2f}%'.format(x*100)).values,textposition = 'outside',)

layout = go.Layout(
    title='点击次数排行榜<br>总点击数:{}  设备数:{}'.format(len(total_appclick), imeis),
    height=12000,
    width=1200,
    margin=dict(l=300,r=0),
    yaxis = dict(ticklen=3)
)

fig = go.Figure(data=[trace],layout=layout)
#py.iplot(fig)
py.plot(fig,image='png', filename='click.html', 
            image_height=12000, image_width=1200, image_filename='click')

'file:///Users/mmuunn/Documents/Works/Userbehavior/click.html'

In [13]:
click_by_phone = total_appclick.groupby(['attrs_packagename']).size().sort_values().truediv(imeis)

click_by_phone = click_by_phone.apply(lambda x: round(x,1))

trace = go.Bar(y=click_by_phone.index, x=click_by_phone.values, orientation = 'h',
                  text = click_by_phone.values,textposition = 'outside',)

layout = go.Layout(
    title='人均点击排名<br>总点击数:{}  设备数:{}'.format(len(total_appclick), imeis),
    height=12000,
    width=1000,
    margin=dict(l=300),
    yaxis = dict(ticklen=3)
)

fig = go.Figure(data=[trace],layout=layout)
#py.iplot(fig)
py.plot(fig,image='png', filename='mclick.html', 
            image_height=12000, image_width=1200, image_filename='mclick')

'file:///Users/mmuunn/Documents/Works/Userbehavior/mclick.html'

In [71]:
def install_rate(df, label,title,height=4000,width=1200):
    data = df[df.name == label].groupby(['IMEI', 'attrs_packagename']).size()
    package_info = data.reset_index()
    total = package_info.groupby('attrs_packagename').size().sort_values().sum()
    package_rate = package_info.groupby('attrs_packagename').size().sort_values()/imeis
    trace = go.Bar(y=package_rate.index, x=package_rate.values, orientation = 'h',
                      text = package_rate.apply(lambda x:'{:.2f}%'.format(x*100)).values,textposition = 'outside',)
    layout = go.Layout(
        title= title + '<br>总次数:{}  总设备数:{}'.format(str(int(total)), imeis),
        height=height,
        width=width,
        margin=dict(l=300,r=0),
        yaxis = dict(ticklen=10)
    )
    fig = go.Figure(data=[trace],layout=layout)
    #py.iplot(fig)
    py.plot(fig,image='png', filename='{}.html'.format(label), 
            image_height=height, image_width=width, image_filename=label)

In [23]:
data = valid_data[valid_data.name == 'appremove'].groupby(['IMEI', 'attrs_packagename']).size()

In [72]:
install_rate(valid_data, 'appinstall', '安装应用覆盖率', height=16000)

In [25]:
install_rate(valid_data,'appremove', '卸载应用排行榜',height=8000)

### Camera 应用 

In [26]:
chinese_name_photos = {'photo':'普通拍照', 'panorama':'全景', 'facebeauty':'美颜', 'hdr':'逆光', 'nightvideo':'夜景',
                'selfiepanorama':'全景自拍', 'polaroid':'心情照片','gifshot':'GIF动画','profession':'高级', 'arcfilter':'滤镜'}
chinese_name_videos = {'video':'普通摄像','slowmotion':'慢动作摄影', 'timelapse':'延时摄影'}

In [27]:
takephoto_data = valid_data[valid_data['name'] == 'TakePhoto']

In [47]:
video_data = valid_data[valid_data['name'] == 'Video']

In [48]:
total_camera = len(takephoto_data)
total_video = len(video_data)
total_used = (len(takephoto_data) + len(video_data))

In [39]:
capturemode = takephoto_data.groupby(['attrs_Capturemode']).size().sort_values()

In [53]:
capture_dict = {}

for p in chinese_name_photos:
    if p in capturemode.index:
        capture_dict[chinese_name_photos[p]] = capturemode[p]
    else:
        capture_dict[chinese_name_photos[p]] = 0

In [54]:
camera_data = pd.Series(capture_dict).sort_values()

In [55]:
trace = go.Bar(y=camera_data.index, x=camera_data.values, orientation = 'h',
                 text = camera_data.apply(lambda x:'{:.1f}%'.format(x*100/total_camera)).values,textposition = 'outside',)

layout = go.Layout(
    title='拍照使用排名<br>总使用次数:{}'.format(total_camera),
    height=400,
    width=1200,
    margin=dict(l=120),
    yaxis = dict(ticklen=10)
)

fig = go.Figure(data=[trace],layout=layout)
py.iplot(fig)

In [44]:
video_mode = video_data.groupby(['attrs_Videomode']).size().sort_values()
video_dict = {}
for p in chinese_name_videos:
    if p in video_mode.index:
        video_dict[chinese_name_videos[p]] = video_mode[p]
        capture_dict[chinese_name_videos[p]] = video_mode[p]
    else:
        capture_dict[chinese_name_videos[p]] = 0
        video_dict[chinese_name_videos[p]] = 0


In [50]:
video_chinese_data = pd.Series(video_dict).sort_values()
trace = go.Bar(y=video_chinese_data.index, x=video_chinese_data.values, orientation = 'h',
                  text = video_mode.apply(lambda x:'{:.1f}%'.format(x*100/total_video)).values,textposition = 'outside',)

layout = go.Layout(
    title='VideoMode排名<br>总使用次数:{}'.format(total_video),
    height=250,
    width=1200,
    margin=dict(l=120),
    yaxis = dict(ticklen=10)
)

fig = go.Figure(data=[trace],layout=layout)
py.iplot(fig)

In [51]:
camera_data = pd.Series(capture_dict).sort_values()

In [52]:
trace = go.Bar(y=camera_data.index, x=camera_data.values, orientation = 'h',
                 text = camera_data.apply(lambda x:'{:.1f}%'.format(x*100/total_used)).values,textposition = 'outside',)

layout = go.Layout(
    title='相机使用排名<br>总使用次数:{}'.format(total_used),
    height=400,
    width=1200,
    margin=dict(l=120),
    yaxis = dict(ticklen=10)
)

fig = go.Figure(data=[trace],layout=layout)
py.iplot(fig)

### 销量统计

In [ ]:
valid_data.groupby(['IMEI', 'wallTime']).size()

In [57]:
register = valid_data.groupby(['IMEI', 'registerTime']).size()

In [58]:
register_data = {}

In [59]:
for imei, group in register.reset_index().groupby(['IMEI']):
    register_date = datetime.datetime.fromtimestamp(group.iloc[-1]['registerTime'])
    register_data[imei] = register_date

In [60]:
psd = pd.DataFrame.from_dict(register_data, orient='index')

In [61]:
len(psd.index)

256

In [62]:
register_count = psd.groupby([0]).size()

In [63]:
cc = register_count.resample('D').sum().fillna(0)

In [64]:
paper_bgcolor='#242640'
plot_bgcolor='#242640'
paper_bgcolor='#FFFFFF'
plot_bgcolor='#FFFFFF'
text_color = '#000000'
x_color = '#000000'
y_color = '#000000'
title_color = '#000000'
legend_color = '#000000'

In [65]:
bandxaxis = go.XAxis(
        type='date',
        #range=[(created_by_day.index[start] - 1).strftime('%y-%m-%d'),
        #       (created_by_day.index[-1] + 1).strftime('%y-%m-%d'),],
        ticks="outside", 
        showticklabels=True,
        ticktext=list(cc.index.strftime('%m-%d')),
        tickvals=list(cc.index.strftime('%y-%m-%d')),
        tickfont=dict(size=11),
        #tickfont=dict(size=11,color='#B9B9C3'),
        ticklen=10,showgrid=True,
        #color='#FFFFFF'
    )

In [74]:
data=[go.Bar(
        x = cc.index,
        y = cc.values,
        text=cc.values, 
        textposition='outside',
        name = '每日提交数',)
        #marker=dict(color='#7D858C')), 
]


In [75]:
layout = go.Layout(
    title='销量趋势图<br>总数:' + str(imeis),
    titlefont=dict(color=title_color,size=25),
    height=1200,
    width=1600,
    bargap=0.1,
    xaxis = bandxaxis,
    yaxis = dict(ticklen=10),
    #bargroupgap=0.05,
    #paper_bgcolor=paper_bgcolor,
    #plot_bgcolor=plot_bgcolor,
    legend=dict(
        x=0.1,
        font=dict(size=16),
    )
)

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)